In [10]:
import pandas as pd
import cudf
%cd /data/dhk/physionet.org/files/mimiciv/2.2/hosp

/data/dhk/physionet.org/files/mimiciv/2.2/hosp


In [11]:
# proodh = pd.read_csv("process_dohun.csv")
df_dh = cudf.read_csv("process_dohun.csv")

df_dh = df_dh.drop('subject_id', axis=1)
df_dh = df_dh.drop('hadm_id', axis=1)
df_dh

,admit_age,gender,insurance,language,marital_status,race,hospital_expire_flag,medication,frequency,drug
0,-0.928571,1,2,1,1,28,0,50,95,34
1,-0.071429,0,2,1,1,28,0,806,16,860
2,-2.071429,0,2,1,1,28,0,50,95,72
3,-0.571429,1,2,1,2,28,0,15,16,1786
4,-1.000000,1,2,1,0,28,0,458,91,503
...,...,...,...,...,...,...,...,...,...,...
10350291,-0.785714,1,2,1,2,28,1,1552,121,1659
10350292,-0.785714,1,2,1,2,28,1,1552,121,1659
10350293,-0.785714,1,2,1,2,28,1,1552,121,1659
10350294,-0.785714,1,2,1,2,28,1,1552,121,1659


In [12]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from cuml.model_selection import train_test_split  
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# model = AdaBoostClassifier()

y = df_dh['hospital_expire_flag'] 
X = df_dh.drop('hospital_expire_flag', axis=1) # 'label' 열을 제외한 나머지 열을 X로 사용


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8,random_state=42)

X_train = X_train.to_cupy()
X_test = X_test.to_cupy()
y_train = y_train.to_cupy()
y_test = y_test.to_cupy()

X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

# X_train = X_train.to_numpy()
# X_test = X_test.to_numpy()
# y_train = y_train.to_numpy()
# y_test = y_test.to_numpy()

# 매개변수 그리드 정의
param = {
  'learning_rate': [0.1, 0.01, 0.001],
  'n_estimators': [100, 500, 1000],
  'learning_rate': [0.1, 0.01, 0.001],
  'n_estimators': [100, 500, 1000],
  'estimator': [DecisionTreeClassifier(max_depth=3), DecisionTreeClassifier(max_depth=3)],
  'algorithm': ['SAMME', 'SAMME.R']
}


# AdaBoostClassifier 생성
ada_boost = AdaBoostClassifier()


# GridSearchCV 객체 생성
grid_search = GridSearchCV(ada_boost, param_grid=param, cv=5, refit=True, n_jobs=-1, return_train_score=True, verbose=2, error_score='raise')

# 훈련 데이터로 모델 학습 및 최적 매개변수 탐색
grid_search.fit(X_train.get(), y_train.get())

# 최적 매개변수 출력
print("최적 매개변수:", grid_search.best_params_)
# 최적 매개변수로 모델 재학습
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

y_pred = best_model.predict(test)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME

[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.001, n_estimators=100; total time=219.4min
[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.001, n_estimators=1000; total time=804.2min
[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.001, n_estimators=1000; total time=233.1min
[CV] END algorithm=SAMME.R, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.1, n_estimators=500; total time=124.9min
[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.1, n_estimators=100; total time=215.7min
[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.001, n_estimators=100; total time=212.6min
[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.1, n_estimators=100; total time=189.3min
[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learn

/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.1, n_estimators=500; total time=863.9min
[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.1, n_estimators=1000; total time=266.7min
[CV] END algorithm=SAMME.R, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.01, n_estimators=1000; total time=247.4min
[CV] END algorithm=SAMME.R, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.001, n_estimators=1000; total time=166.0min
[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.01, n_estimators=1000; total time=1016.6min
[CV] END algorithm=SAMME, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.001, n_estimators=500; total time=118.3min
[CV] END algorithm=SAMME.R, estimator=DecisionTreeClassifier(max_depth=3), learning_rate=0.01, n_estimators=1000; total time=247.2min
[CV] END algorithm=SAMME.R, estimator=DecisionTreeClassifier(max_depth=

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [14]:
y_pred = best_model.predict(X_test.get())

In [15]:
import cuml
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

cu_score = cuml.metrics.accuracy_score(y_test, y_pred )
print(cu_score)

# 정확도 계산
accuracy = accuracy_score(y_test.get(), y_pred)
print("Accuracy:", accuracy)

# F1 스코어 계산
f1 = f1_score(y_test.get(), y_pred)
print("F1 Score:", f1)

0.8790247440338135
Accuracy: 0.8790247625672686
F1 Score: 0.8838944365060305


In [16]:
import pandas as pd

# randomForest_y_pred를 Pandas DataFrame으로 변환
y_pred_df = pd.DataFrame(y_pred)

# CSV 파일로 저장
y_pred_df.to_csv('adaboost_pred.csv', index=False, header=False)